In [0]:
print('You are great')

You are great


In [0]:
%sql
create database ecommerce;

In [0]:
# For Customer Details (Dimension table)
day1='dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_01_10_24.csv'
day2='dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_02_10_24.csv'
day3='dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_03_10_24.csv'
import pandas as pd
from pyspark.sql.functions import col,to_date
from delta.tables import *
from pyspark.sql.functions import col, lit, current_date

#my_schema= "customer_id STRING, name STRING, state STRING, city STRING, address STRING, mb_number LONG, pin INT, valid_from date, valid_to date"

# Step 1 - Creating New df on the arrival of new file in the loacaiton we mentioned in the load section
date='01_10_24'
# date='02_10_24'
# date='03_10_24'

#Ideally here we are providing date manually whereas when we will push this code and run the notebook on daily basis then we can use date function so that ever data our data got change and it will pick that day's file

customer_df=spark.read.format('csv').option('header',True).option('inferSchema',True).load(f'dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_{date}.csv')

#customer_df=spark.read.format('csv').option('header',True).schema(my_schema).load(f'{day1}')
# customer_df=df.withColumn('valid_from1',col('valid_from').cast('date')).withColumn('valid_to1',col('valid_to').cast('date'))

customer_df.printSchema()
customer_df.show()

# Here we will use if condition so if that table will already be existing then we will use append otherwise we will simple do overwirte
customer_path='ecommerce.customer_dimension'

if spark.catalog.tableExists(customer_path):
    print('table exist')
    # Load existing customer_dimension table so that we can update this table with new incremental data and delete its old image
    customer_dim_table = DeltaTable.forPath(spark, 'dbfs:/user/hive/warehouse/ecommerce.db/customer_dimension')

    # In this merge condition if the primary key value already existing then we will simply update its value with new value but if the primary key does not exist then we will simple insert the data.
    customer_dim_table.alias("dim") \
        .merge(
            customer_df.alias("raw"),
            "dim.customer_id = raw.customer_id"
        ) \
        .whenMatchedUpdate(set={
            "customer_id": "raw.customer_id",
            "name": "raw.name",
            "state": "raw.state",
            "city": "raw.city",
            "address": "raw.address",
            "mb_number": "raw.mb_number",
            "pin": "raw.pin",
            "valid_from": "raw.valid_from",
            "valid_to": "raw.valid_to"
            
        }) \
        .whenNotMatchedInsert(values={
            "customer_id": "raw.customer_id",
            "name": "raw.name",
            "state": "raw.state",
            "city": "raw.city",
            "address": "raw.address",
            "mb_number": "raw.mb_number",
            "pin": "raw.pin",
            "valid_from": "raw.valid_to"  # Future date for new records
        }) \
        .execute()
else:
    # In this else condition if the table does not existin then here it will create the table
    #more specifically creatig delta table 
    customer_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.customer_dimension')


# In this blow we are maintaing only 1 table we are not creating any staging because this is dimension table so there won't be much data transaction in this table
#Loading incremental data from source and creating df on it then we are using merge to update the dimension table.




root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- address: string (nullable = true)
 |-- mb_number: long (nullable = true)
 |-- pin: integer (nullable = true)
 |-- valid_from: string (nullable = true)
 |-- valid_to: string (nullable = true)

+-----------+----------------+----------------+-----------+----------------+----------+------+----------+----------+
|customer_id|            name|           state|       city|         address| mb_number|   pin|valid_from|  valid_to|
+-----------+----------------+----------------+-----------+----------------+----------+------+----------+----------+
|       C001|        John Doe|   Uttar Pradesh|  Ghaziabad|     Indirapuram|9876543210|201010| 10/1/2024|12/12/9999|
|       C002|      Jane Smith|           Delhi|      Delhi| Connaught Place|8765432109|110001| 10/1/2024|12/12/9999|
|       C003| Michael Johnson|     Maharashtra|     Mumbai|  

In [0]:
# # Define the file paths for three days
# day1 = 'dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_01_10_24.csv'
# day2 = 'dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_02_10_24.csv'
# day3 = 'dbfs:/FileStore/Ecommerce_data_02_10_24/Customer_details_03_10_24.csv'

# # Correct schema definition
# from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DateType

# my_schema = StructType([
#     StructField("customer_id", StringType(), True),
#     StructField("name", StringType(), True),
#     StructField("state", StringType(), True),
#     StructField("city", StringType(), True),
#     StructField("address", StringType(), True),
#     StructField("mb_number", LongType(), True),
#     StructField("pin", IntegerType(), True),
#     StructField("valid_from", DateType(), True),
#     StructField("valid_to", DateType(), True)
# ])

# # Load the data from day1 CSV with the schema
# customer_df = spark.read.format('csv').option('header', 'true').schema(my_schema).load(f'{day1}')

# # Display the schema and the data
# customer_df.printSchema()
# customer_df.show()


In [0]:
# For Customer Details (Dimension table)
# Here we are dealing with 2 table
# Fact table of order (order_raw)
# Fact table  joining order table and customer dimension table (join_table)


# Here also we are using date logic to pick present day data 
day1='dbfs:/FileStore/Ecommerce_data_02_10_24/Order_details_01_10_24.csv'
day2='dbfs:/FileStore/Ecommerce_data_02_10_24/Order_details_02_10_24.csv'
day3='dbfs:/FileStore/Ecommerce_data_02_10_24/Order_details_03_10_24.csv'
import pandas as pd
from pyspark.sql.functions import col,to_date
from delta.tables import *
from pyspark.sql.functions import col, lit, current_date

date='01_10_24'
# date='02_10_24'
# date='03_10_24'

order_df=spark.read.format('csv').option('header',True).option('inferSchema',True).load(f'dbfs:/FileStore/Ecommerce_data_02_10_24/Order_details_{date}.csv')
#order_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.order_raw')

# Here we are again using if condition that if table does not exist then it will go in else condition to create table when this pipeline run for the first time

# But if table already existi then we are simply load incremental data into df then merge into existing table to update the old data on the basis of primary key
table_path='ecommerce.order_raw'
if spark.catalog.tableExists(table_path):
    print('Table alredy exist')
    order_fact=DeltaTable.forPath(spark,'dbfs:/user/hive/warehouse/ecommerce.db/order_raw')
    order_fact.alias('dim')\
        .merge(
            order_df.alias('raw'),
            'dim.order_id=raw.order_id'
        )\
        .whenMatchedUpdate(set={
            "order_id":"raw.order_id",
            "customer_id":"raw.customer_id",
            "product":"raw.product",
            "qty":"raw.qty",
            "order_date":"raw.order_date",
            "status":"raw.status",
            "update_date":"raw.update_date"

        })\
            .whenNotMatchedInsert(values={
                "order_id":"raw.order_id",
            "customer_id":"raw.customer_id",
            "product":"raw.product",
            "qty":"raw.qty",
            "order_date":"raw.order_date",
            "status":"raw.status",
            "update_date":"raw.update_date"
                }).execute()



else:
    order_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.order_raw')


# Here we are creating df on customer dimension table so that we can use to in our join condition
df_customer=spark.read.format('delta').load('dbfs:/user/hive/warehouse/ecommerce.db/customer_dimension')

# This will help to join incremental order data with customer dimension data
df_join=order_df.join(df_customer,'customer_id')

# Again using if else condition to create table for the first time and then using merge condition to update the odl data with new data on the basis of primary key
table_path2='ecommerce.order_join'
if spark.catalog.tableExists(table_path2):
    print('Joined table exist')
    
    order_join=DeltaTable.forPath(spark,'dbfs:/user/hive/warehouse/ecommerce.db/order_join')
    
    df_join.printSchema()
   
    order_join.alias('table_join')\
        .merge(df_join.alias('df_join'),
               'table_join.order_id=df_join.order_id')\
        .whenMatchedUpdate(set={
            "order_id":"df_join.order_id",
            'ingestion_type':lit("updated_data"),
            "product":"df_join.product",
            "qty":"df_join.qty",
            "order_date":"df_join.order_date",
            "status":"df_join.status",
            "update_date":"df_join.update_date",
            "customer_id": "df_join.customer_id",
            "name": "df_join.name",
            "state": "df_join.state",
            "city": "df_join.city",
            "address": "df_join.address",
            "mb_number": "df_join.mb_number",
            "pin": "df_join.pin",
            "valid_from": "df_join.valid_from",
            "valid_to": "df_join.valid_to"

        })\
        .whenNotMatchedInsert(values={"order_id":"df_join.order_id",
            "customer_id":"df_join.customer_id",
            "product":"df_join.product",
            "qty":"df_join.qty",
            "order_date":"df_join.order_date",
            "status":"df_join.status",
            "update_date":"df_join.update_date",
            'ingestion_type':lit("inserted_data"),
            "name": "df_join.name",
            "state": "df_join.state",
            "city": "df_join.city",
            "address": "df_join.address",
            "mb_number": "df_join.mb_number",
            "pin": "df_join.pin",
            "valid_from": "df_join.valid_from",
            "valid_to": "df_join.valid_to"
            }).execute()
    

else:
    # lit is using to provide a string value to the column
    df_join=df_join.withColumn('ingestion_type',lit('insert_data'))
    df_join.write.format('delta').mode('overwrite').saveAsTable('ecommerce.order_join')





order_df.show()
df_join.printSchema()
df_join.show()

+--------+-----------+---------------+---+-----+----------+---------+-----------+
|order_id|customer_id|        product|qty|price|order_date|   status|update_date|
+--------+-----------+---------------+---+-----+----------+---------+-----------+
|    O001|       C001|         Laptop|  1|50000|2024-09-01|  Shipped|   1-Oct-24|
|    O002|       C002|     Smartphone|  2|30000|2024-09-05|Delivered|   1-Oct-24|
|    O003|       C001|     Headphones|  3| 5000|2024-09-10|Cancelled|   1-Oct-24|
|    O004|       C003|         Tablet|  1|25000|2024-09-15|  Shipped|   1-Oct-24|
|    O005|       C005|         Camera|  2|40000|2024-09-20|Delivered|   1-Oct-24|
|    O006|       C006|          Watch|  1|20000|2024-09-25|  Shipped|   1-Oct-24|
|    O007|       C007|             TV|  1|35000|2024-09-30|Delivered|   1-Oct-24|
|    O008|       C008|   Refrigerator|  1|25000|2024-10-05|  Shipped|   1-Oct-24|
|    O009|       C009|Air Conditioner|  2|30000|2024-10-10|Delivered|   1-Oct-24|
|    O010|      

In [0]:
#To delete any directory
#dbutils.fs.rm("dbfs:/FileStore/Ecommerce_data_02_10_24", recurse=True)


In [0]:
# %sql
# drop table ecommerce.order_join;
# --select * from ecommerce.order_join;

In [0]:
# %sql
# -- All the 3 tables
# select * from  ecommerce.order_raw;
# select * from  ecommerce.customer_dimension;
# select * from  ecommerce.order_join;

# -- drop table ecommerce.order_raw;
# -- drop table ecommerce.customer_dimension;
# -- drop table ecommerce.order_join;

In [0]:
# Aggrigation Price  on the basis state for only Shipped and Delivered Order(excluding Cancelled POs)
#msitaker i was doing - i was trying to do aggregation without importing sum functionality

from pyspark.sql.functions import sum , count
sales_df=spark.read.format('delta').option('infer_schema',True).load('dbfs:/user/hive/warehouse/ecommerce.db/order_join')
sales_df.printSchema()

amount_state_agg=sales_df.filter(col('status').isin('Shipped','Delivered')).groupBy('state').agg(sum('price').alias('total_spent')).orderBy('total_spent')

amount_state_agg.write.format('delta').mode('overwrite').saveAsTable('ecommerce.sales_per_city')
amount_state_agg.show()


root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- product: string (nullable = true)
 |-- qty: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- status: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- address: string (nullable = true)
 |-- mb_number: long (nullable = true)
 |-- pin: integer (nullable = true)
 |-- valid_from: string (nullable = true)
 |-- valid_to: string (nullable = true)
 |-- ingestion_type: string (nullable = true)

+--------------+-----------+
|         state|total_spent|
+--------------+-----------+
|        Punjab|       5000|
|Madhya Pradesh|      10000|
|       Haryana|      10000|
|        Odisha|      15000|
|        Kerala|      15000|
|       Gujarat|      20000|
|   West Bengal|      20000|
|   Maharashtra|      25000|
|     T

In [0]:
# %sql
# select * from ecommerce.order_join;

In [0]:
# Aggrigate quantity on the basis of product to find which product is selling the most

total_product_quantity_df=sales_df.filter(col('status').isin('Shipped','Delivered')).groupBy('product').agg(sum('qty').alias('total_quantity')).orderBy('total_quantity')

total_product_quantity_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.total_produce_quantity')

total_product_quantity_df.show()


+---------------+--------------+
|        product|total_quantity|
+---------------+--------------+
|             TV|             1|
|           Oven|             1|
|   Refrigerator|             1|
|         Laptop|             1|
| Vacuum Cleaner|             1|
|   Coffee Maker|             1|
|Washing Machine|             1|
|         Tablet|             1|
|          Watch|             1|
|  Mixer Grinder|             2|
|      Microwave|             2|
|Air Conditioner|             2|
|         Camera|             2|
|     Smartphone|             2|
+---------------+--------------+



In [0]:
shipped_data_df=sales_df.filter(col('status')=='Shipped')
shipped_data_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.shipped_order_table')

cancelled_data_df=sales_df.filter(col('status')=='Cancelled')
cancelled_data_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.cancelled_order_table')

delivered_data_df=sales_df.filter(col('status')=='Delivered')
delivered_data_df.write.format('delta').mode('overwrite').saveAsTable('ecommerce.delivered_order_table')


In [0]:
%sql
select * from  ecommerce.shipped_order_table;


customer_id,order_id,product,qty,price,order_date,status,update_date,name,state,city,address,mb_number,pin,valid_from,valid_to,ingestion_type
C001,O001,Laptop,1,50000,2024-09-01,Shipped,1-Oct-24,John Doe,Uttar Pradesh,Ghaziabad,Indirapuram,9876543210,201010,10/1/2024,12/12/9999,insert_data
C003,O004,Tablet,1,25000,2024-09-15,Shipped,1-Oct-24,Michael Johnson,Maharashtra,Mumbai,Andheri,7654321098,400059,10/1/2024,12/12/9999,insert_data
C006,O006,Watch,1,20000,2024-09-25,Shipped,1-Oct-24,Sarah Lee,West Bengal,Kolkata,Park Street,4321098765,700016,10/1/2024,12/12/9999,insert_data
C008,O008,Refrigerator,1,25000,2024-10-05,Shipped,1-Oct-24,Olivia Taylor,Telangana,Hyderabad,Hitech City,2109876543,500081,10/1/2024,12/12/9999,insert_data
C010,O010,Washing Machine,1,20000,2024-10-15,Shipped,1-Oct-24,Ava Miller,Gujarat,Ahmedabad,Ellis Bridge,9876543210,380006,10/1/2024,12/12/9999,insert_data
C012,O012,Coffee Maker,1,10000,2024-10-25,Shipped,1-Oct-24,Sophia Davis,Haryana,Gurgaon,Golf Course Road,7654321098,122002,10/1/2024,12/12/9999,insert_data
C014,O014,Microwave,2,10000,2024-11-05,Shipped,1-Oct-24,Lily Harris,Madhya Pradesh,Bhopal,Lake View,5432109876,462001,10/1/2024,12/12/9999,insert_data


In [0]:
%sql
select * from  ecommerce.cancelled_order_table;


customer_id,order_id,product,qty,price,order_date,status,update_date,name,state,city,address,mb_number,pin,valid_from,valid_to,ingestion_type
C001,O003,Headphones,3,5000,2024-09-10,Cancelled,1-Oct-24,John Doe,Uttar Pradesh,Ghaziabad,Indirapuram,9876543210,201010,10/1/2024,12/12/9999,insert_data


In [0]:
%sql
select * from  ecommerce.delivered_order_table;

customer_id,order_id,product,qty,price,order_date,status,update_date,name,state,city,address,mb_number,pin,valid_from,valid_to,ingestion_type
C002,O002,Smartphone,2,30000,2024-09-05,Delivered,1-Oct-24,Jane Smith,Delhi,Delhi,Connaught Place,8765432109,110001,10/1/2024,12/12/9999,insert_data
C005,O005,Camera,2,40000,2024-09-20,Delivered,1-Oct-24,David Wilson,Karnataka,Bengaluru,Whitefield,5432109876,560066,10/1/2024,12/12/9999,insert_data
C007,O007,TV,1,35000,2024-09-30,Delivered,1-Oct-24,Thomas Brown,Andhra Pradesh,Hyderabad,Banjara Hills,3210987654,500034,10/1/2024,12/12/9999,insert_data
C009,O009,Air Conditioner,2,30000,2024-10-10,Delivered,1-Oct-24,Ethan Green,Rajasthan,Jaipur,Pink City,1098765432,302025,10/1/2024,12/12/9999,insert_data
C011,O011,Mixer Grinder,2,5000,2024-10-20,Delivered,1-Oct-24,Benjamin Johnson,Punjab,Chandigarh,Sector 17,8765432109,160017,10/1/2024,12/12/9999,insert_data
C013,O013,Oven,1,15000,2024-10-30,Delivered,1-Oct-24,William Wilson,Kerala,Kochi,Fort Kochi,6543210987,682001,10/1/2024,12/12/9999,insert_data
C015,O015,Vacuum Cleaner,1,15000,2024-11-10,Delivered,1-Oct-24,Noah Carter,Odisha,Bhubaneswar,Janpath,4321098765,751001,10/1/2024,12/12/9999,insert_data


In [0]:
# sales_df=spark.read.format('delta').option('infer_schema',True).load('dbfs:/user/hive/warehouse/ecommerce.db/order_join')
# sales_df.show()

+-----------+--------+---------------+---+-----+----------+---------+-----------+----------------+--------------+-----------+----------------+----------+------+----------+----------+--------------+
|customer_id|order_id|        product|qty|price|order_date|   status|update_date|            name|         state|       city|         address| mb_number|   pin|valid_from|  valid_to|ingestion_type|
+-----------+--------+---------------+---+-----+----------+---------+-----------+----------------+--------------+-----------+----------------+----------+------+----------+----------+--------------+
|       C001|    O003|     Headphones|  3| 5000|2024-09-10|Cancelled|   1-Oct-24|        John Doe| Uttar Pradesh|  Ghaziabad|     Indirapuram|9876543210|201010| 10/1/2024|12/12/9999|   insert_data|
|       C001|    O001|         Laptop|  1|50000|2024-09-01|  Shipped|   1-Oct-24|        John Doe| Uttar Pradesh|  Ghaziabad|     Indirapuram|9876543210|201010| 10/1/2024|12/12/9999|   insert_data|
|       C0

In [0]:
#dbutils.fs.rm("dbfs:/user/hive/warehouse/ecommerce.db", recurse=True)